In [ ]:
from data.load_cifrar100 import *

train_loader, val_loader, test_loader = get_cifar100_dataloaders(
    batch_size=64,
    data_dir="./data",
    num_workers=4,
    val_split=0.1,
    img_size=32 , seed=77)

---


In [ ]:
from src.Model_A_OutGridNet import * 
from src.stage_config import *

from src.training.one_epoch_train import *
from src.training.train_full_model import *

# ---------------------------------------------------------
# MODELO 1: HÍBRIDO COMPLETO (Baseline)
# ---------------------------------------------------------
def configs_hybrid():
    return [
        StageCfg(dim=64,  depth=2, num_heads=2, grid_size=8, outlook_heads=2, drop_path=0.0),
        StageCfg(dim=128, depth=2, num_heads=4, grid_size=8, outlook_heads=4, drop_path=0.0),
        StageCfg(dim=256, depth=3, num_heads=8, grid_size=4, outlook_heads=8, drop_path=0.0),
        StageCfg(dim=384, depth=1, num_heads=6, grid_size=2, outlook_heads=6, drop_path=0.0),]

experiments = [
    {
        "name": "Hybrid_Full",
        "stages_fn": configs_hybrid,
        "dpr": 0.1,     
        "desc": "Baseline Completo"
    },]


def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

device = "cuda" if torch.cuda.is_available() else "cpu"

# Loop de entrenamiento
for exp in experiments:
    print(f"\n{'='*40}")
    print(f"Iniciando: {exp['name']} ({exp['desc']})")
    print(f"Drop Path Rate: {exp['dpr']}")
    print(f"{'='*40}")

    # Instanciar Modelo LIMPIO
    stages = exp["stages_fn"]()
    model = MaxOutNet(
        num_classes=100,
        stages=stages,
        stem_dim=64,
        dpr_max=exp["dpr"]).to(device)

    n_params = count_trainable_parameters(model)
    print(f"Trainable parameters: {n_params:,}")

    # Entrenar 100 Epocas
    history, _ = train_model(
        model=model,
        train_loader=train_loader,
        epochs=100,
        val_loader=val_loader,
        device=device,

        lr=5e-4,
        weight_decay=0.05,

        # Mixed Precision
        autocast_dtype="bf16" if device == "cuda" else "fp32",
        use_amp=(device == "cuda"),
        grad_clip_norm=1.0,

        warmup_ratio=0.05,
        min_lr=1e-6,

        label_smoothing=0.1,

        print_every=600,
        save_path=f"best_{exp['name']}.pt",
        last_path=f"last_{exp['name']}.pt",

        mix_prob=0.5,
        mixup_alpha=0.8,
        cutmix_alpha=1.0,

        num_classes=100,
        channels_last=True)

    print()
    print('Test del Modelo')
    free_all_cuda("model", "optimizer", "scaler", "scheduler", "batch", "loss", "outputs", "logits")


Iniciando: Hybrid_Full (Baseline Completo)
Drop Path Rate: 0.1
Trainable parameters: 14,599,198
=== Run config ===
device=cuda | amp=True | autocast_dtype=bf16 | channels_last=True
epochs=100 | steps/epoch=704 | total_steps=70400 | warmup_steps=3520
batch_size=64 | input_shape=(64, 3, 32, 32) | num_classes=100
opt=AdamW | lr=0.0005 | wd=0.05 | grad_clip_norm=1.0
aug: mix_prob=0.5 | mixup_alpha=0.8 | cutmix_alpha=1.0 | label_smoothing=0.1
early_stop=True | metric=top1 | patience=12 | min_delta=0.05

=== Epoch 1/100 ===
[train step 600/704] loss 4.2991 | top1 5.68% | top3 13.62% | top5 19.28% | 665.3 img/s | lr 8.52e-05 | gnorm 5.746 | clip 100.0% | oflow 0 | nonfinite 0 | scale 1.0
[train step 704/704] loss 4.2481 | top1 6.40% | top3 15.08% | top5 21.13% | 663.2 img/s | lr 1.00e-04 | gnorm 5.707 | clip 100.0% | oflow 0 | nonfinite 0 | scale 1.0
[Train] loss 4.2481 | top1 6.40% | top3 15.08% | top5 21.13% | lr 1.00e-04 | grad_norm 5.707 | clip 100.0% | amp_overflows 0 | nonfinite_loss 0

TypeError: string indices must be integers, not 'str'

In [ ]:
evaluate_one_epoch(model=model,dataloader=test_loader)

(0.828276930809021, {'top1': 79.85, 'top3': 91.74, 'top5': 94.9})

---